In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import  Session, QiskitRuntimeService, RuntimeDecoder, EstimatorV2 as Estimator
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram

from qiskit_aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

import numpy as np
import matplotlib.pyplot as plt
import json

from qiskit.quantum_info import Pauli, PauliList
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit.circuit.instruction import Instruction
from qiskit.circuit.quantumcircuitdata import CircuitInstruction
from qiskit.result import marginal_counts, marginal_distribution

import warnings



In [ ]:
noise_model = NoiseModel()

p0given1 = 0.07
p1given0 = 0.02

error_meas=ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])

for i in range(8):
    p_reset = 0.02*i+0.01
    error_reset_i = pauli_error([('X', p_reset), ('I', 1 - p_reset)]) #use the reset error as the state-preparation errors
    noise_model.add_quantum_error(error_reset_i,"reset",[i])

noise_model.add_all_qubit_readout_error(error_meas,"measure")

print(noise_model)

sim_be = AerSimulator(method='density_matrix',noise_model=noise_model)


NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx']
  Instructions with noise: ['measure', 'reset']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7]
  All-qubits errors: ['measure']
  Specific qubit errors: [('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('reset', (4,)), ('reset', (5,)), ('reset', (6,)), ('reset', (7,))]


In [4]:
def AddingInitialZ(tarCirc, qIdxList):
    modified_data = []
    return_circ = tarCirc.copy()
    
    qList=return_circ.qubits
    for qIdx in qIdxList:
        modified_data.append(CircuitInstruction(Instruction(name='z', num_qubits=1, num_clbits=0, params=[]), [qList[qIdx]], []))
    
    modified_data+=return_circ.data
    return_circ.data = modified_data
    return return_circ

def AddingInitialReset(tarCirc):
    modified_data = []
    return_circ = tarCirc.copy()
    
    qList=return_circ.qubits
    for qi in qList:
        modified_data.append(CircuitInstruction(Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), [qi], []))
    
    modified_data+=return_circ.data
    return_circ.data = modified_data
    return return_circ

def circuit_CNOTq0q1_Initial00(circ,qList):
    [q0,q1]=qList
    circ.cx(q0,q1)
    return circ

def circuit_CNOTq0q1_Initial10(circ,qList):
    [q0,q1]=qList
    circ.x(q0)
    circ.cx(q0,q1)
    return circ

def SPerrorCircuits(qList):
    L=len(qList) # L must be even
    circ=QuantumCircuit(L,L)
    circEven0=circ.copy(name="CircEven0")
    circEven1=circ.copy(name="CircEven1")
    circOdd0=circ.copy(name="CircOdd0")
    circOdd1=circ.copy(name="CircOdd1")
    for i in range(L//2):
        currentList=qList[i*2:i*2+2]
        circEven0=circuit_CNOTq0q1_Initial00(circEven0,currentList)
        circEven1=circuit_CNOTq0q1_Initial10(circEven1,currentList)
        currentListR=currentList
        currentListR.reverse()
        #print(currentListR)
        circOdd0=circuit_CNOTq0q1_Initial00(circOdd0,currentListR)
        circOdd1=circuit_CNOTq0q1_Initial10(circOdd1,currentListR)

    return [circOdd0,circOdd1, circEven0, circEven1]

def SPerrorCircuitsWithInitialZ(qList):
    L=len(qList)
    circList=SPerrorCircuits(qList)
    oddQList=qList[0::2]
    evenQList=qList[1::2]
    circListNew=[]
    for circ in circList:
        circListNew.append(circ)
        circListNew.append(AddingInitialZ(circ,oddQList))
        circListNew.append(AddingInitialZ(circ,evenQList))
        circListNew.append(AddingInitialZ(circ,qList))


    return circListNew

def Zobservables(qNum):
    zlist=[]
    for i in range(qNum):
        tmpStr="I"*i+"Z"+"I"*(qNum-1-i)
        zlist.append(tmpStr)
    zlist.reverse()
    return zlist

def Dsp_from_Dspam(DspamTilde, Dspam):
    dsp0=(DspamTilde[0]-Dspam[0])/(1-Dspam[0]-Dspam[1])
    dsp1=(DspamTilde[1]-Dspam[1])/(1-Dspam[0]-Dspam[1])

    if dsp0<0 or dsp1<0:
        warnings.warn("Negative D_sp detected, this can happen when gate errors much exceed the state-preparation errors.")

    return (dsp0+dsp1)/2

def DspS_from_Dspam(DspamTilde, Dspam):
    dsp0=(DspamTilde[0]-Dspam[0])/(1-Dspam[0]-Dspam[1])
    dsp1=(DspamTilde[1]-Dspam[1])/(1-Dspam[0]-Dspam[1])

    if dsp0<0 or dsp1<0:
        warnings.warn("Negative D_sp detected, this can happen when gate errors much exceed the state-preparation errors.")

    return [dsp0,dsp1]

def Dm_from_Dspam(dsp, Dspam):
    dm0_plus_dm1=(Dspam[0]+Dspam[1]-2*dsp)/(1-2*dsp)
    dm0_minus_dm1=Dspam[0]-Dspam[1]

    dm0=0.5*(dm0_plus_dm1+dm0_minus_dm1)
    dm1=0.5*(dm0_plus_dm1-dm0_minus_dm1)

    return [dm0, dm1]

def DspSList_from_results(results):
    resOdd0=(1-np.mean(results[0:4],axis=0))/2
    resOdd1=(1+np.mean(results[4:8],axis=0))/2
    resEven0=(1-np.mean(results[8:12],axis=0))/2
    resEven1=(1+np.mean(results[12:16],axis=0))/2
    L=len(resOdd0)
    DspList=[]
    for i in range(L):
        if (i % 2)==0:
            DspamTilde_a=[resEven0[i+1],resEven1[i+1]]
            Dspam_a=[resOdd0[i+1],resOdd1[i+1]]
            dsp=DspS_from_Dspam(DspamTilde_a, Dspam_a)
            DspList.append(dsp)
        else:
            DspamTilde_a=[resOdd0[i-1],resOdd1[i-1]]
            Dspam_a=[resEven0[i-1],resEven1[i-1]]
            dsp=DspS_from_Dspam(DspamTilde_a, Dspam_a)
            DspList.append(dsp)
    

    return DspList

def DspList_from_results(results):
    resOdd0=(1-np.mean(results[0:4],axis=0))/2
    resOdd1=(1+np.mean(results[4:8],axis=0))/2
    resEven0=(1-np.mean(results[8:12],axis=0))/2
    resEven1=(1+np.mean(results[12:16],axis=0))/2
    L=len(resOdd0)
    DspList=[]
    for i in range(L):
        if (i % 2)==0:
            DspamTilde_a=[resEven0[i+1],resEven1[i+1]]
            Dspam_a=[resOdd0[i+1],resOdd1[i+1]]
            dsp=Dsp_from_Dspam(DspamTilde_a, Dspam_a)
            DspList.append(dsp)
        else:
            DspamTilde_a=[resOdd0[i-1],resOdd1[i-1]]
            Dspam_a=[resEven0[i-1],resEven1[i-1]]
            dsp=Dsp_from_Dspam(DspamTilde_a, Dspam_a)
            DspList.append(dsp)
    

    return DspList

def DmList_from_results(results, DspList):
    resOdd0=(1-np.mean(results[0:4],axis=0))/2
    resOdd1=(1+np.mean(results[4:8],axis=0))/2
    resEven0=(1-np.mean(results[8:12],axis=0))/2
    resEven1=(1+np.mean(results[12:16],axis=0))/2
    L=len(resOdd0)
    DmList=[]
    for i in range(L):
        if (i % 2)==0:
            Dspam_t=[resEven0[i],resEven1[i]]
            dsp_t=DspList[i]
            Dm=Dm_from_Dspam(dsp_t, Dspam_t)
            DmList.append(Dm)
        else:
            Dspam_t=[resOdd0[i],resOdd1[i]]
            dsp_t=DspList[i]
            Dm=Dm_from_Dspam(dsp_t, Dspam_t)
            DmList.append(Dm)
    

    return DmList

In [5]:
circuits=SPerrorCircuitsWithInitialZ(list(range(4)))
ii=0
for circ in circuits:
    circNew=AddingInitialReset(circ)
    if ii % 4==0:
        print(circNew.draw())
    ii+=1

     ┌───────┐┌───┐
q_0: ┤ reset ├┤ X ├
     ├───────┤└─┬─┘
q_1: ┤ reset ├──■──
     ├───────┤┌───┐
q_2: ┤ reset ├┤ X ├
     ├───────┤└─┬─┘
q_3: ┤ reset ├──■──
     └───────┘     
c: 4/══════════════
                   
     ┌───────┐     ┌───┐
q_0: ┤ reset ├─────┤ X ├
     ├───────┤┌───┐└─┬─┘
q_1: ┤ reset ├┤ X ├──■──
     ├───────┤└───┘┌───┐
q_2: ┤ reset ├─────┤ X ├
     ├───────┤┌───┐└─┬─┘
q_3: ┤ reset ├┤ X ├──■──
     └───────┘└───┘     
c: 4/═══════════════════
                        
     ┌───────┐     
q_0: ┤ reset ├──■──
     ├───────┤┌─┴─┐
q_1: ┤ reset ├┤ X ├
     ├───────┤└───┘
q_2: ┤ reset ├──■──
     ├───────┤┌─┴─┐
q_3: ┤ reset ├┤ X ├
     └───────┘└───┘
c: 4/══════════════
                   
     ┌───────┐┌───┐     
q_0: ┤ reset ├┤ X ├──■──
     ├───────┤└───┘┌─┴─┐
q_1: ┤ reset ├─────┤ X ├
     ├───────┤┌───┐└───┘
q_2: ┤ reset ├┤ X ├──■──
     ├───────┤└───┘┌─┴─┐
q_3: ┤ reset ├─────┤ X ├
     └───────┘     └───┘
c: 4/═══════════════════
                        


In [14]:
Nqubits=4

circuits=SPerrorCircuitsWithInitialZ(list(range(Nqubits)))

observables = PauliList( Zobservables(Nqubits) )

resultList=[]

for circuit in circuits:
    pm = generate_preset_pass_manager(backend=sim_be, optimization_level=0, initial_layout=range(Nqubits))
    isa_psi = pm.run(AddingInitialReset(circuit))
    isa_observables = [h.apply_layout(isa_psi.layout) for h in observables ]
    
    estimator = Estimator(backend=sim_be)
    
    # calculate [ <psi(theta1)|hamiltonian|psi(theta)> ]
    job = estimator.run([(isa_psi, isa_observables)])
    pub_result = job.result()[0]
    resultList.append(pub_result.data.evs)
    print(f"Expectation values: {pub_result.data.evs}")

Expectation values: [0.87353516 0.89599609 0.75634766 0.84716797]
Expectation values: [0.88574219 0.89892578 0.7734375  0.8359375 ]
Expectation values: [0.89599609 0.90722656 0.74658203 0.84277344]
Expectation values: [0.88330078 0.91162109 0.74853516 0.83544922]
Expectation values: [-0.79199219 -0.81152344 -0.65966797 -0.71875   ]
Expectation values: [-0.78320312 -0.80615234 -0.65185547 -0.72167969]
Expectation values: [-0.78466797 -0.79980469 -0.63720703 -0.71533203]
Expectation values: [-0.78710938 -0.79833984 -0.62744141 -0.72802734]
Expectation values: [0.95214844 0.88818359 0.87109375 0.74414062]
Expectation values: [0.94335938 0.8828125  0.87792969 0.76708984]
Expectation values: [0.94189453 0.89501953 0.86376953 0.76611328]
Expectation values: [0.93505859 0.8828125  0.87597656 0.76318359]
Expectation values: [-0.84472656 -0.78710938 -0.77490234 -0.64013672]
Expectation values: [-0.83837891 -0.79541016 -0.77197266 -0.67041016]
Expectation values: [-0.84130859 -0.79638672 -0.7802

In [15]:
dSPs=DspList_from_results(resultList)
dMs=DmList_from_results(resultList,dSPs)
print(f"Delta SP for the four qubits: {dSPs}")
print(f"Delta M for the four qubits: {dMs}")

Delta SP for the four qubits: [0.008293415314220346, 0.03202542894251145, 0.04718530101641908, 0.07461247496847882]
Delta M for the four qubits: [[0.020912432765926033, 0.07114436635967603], [0.019067747008546278, 0.06881139935229627], [0.02102696437616685, 0.07028233546991686], [0.01137234316428385, 0.07106472597678384]]


In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum") #need input token=""

backend_nazca=service.backend("ibm_nazca")

In [ ]:
Nqubits=2

circuits=SPerrorCircuitsWithInitialZ(list(range(Nqubits)))

observables = PauliList( Zobservables(Nqubits) )

#submit the circuits to backend_nazca via the session method
with Session(service=service, backend=backend_nazca) as session:
    for circuit in circuits:
        pm = generate_preset_pass_manager(backend=backend_nazca, optimization_level=0, initial_layout=[20,33])
        isa_psi = pm.run(circuit)
        isa_observables = [h.apply_layout(isa_psi.layout) for h in observables ]
        
        estimator = Estimator(session=session,backend=backend_nazca,options={"resilience_level": 0,"default_shots": 80000})
        options = estimator.options
        options.resilience.measure_mitigation = False
        options.resilience.zne_mitigation = True
        options.resilience.pec_mitigation = False
        
        job = estimator.run([(isa_psi, isa_observables)])
        #pub_result = job.result()[0]  #comment these two lines if don't need immediate output
        #print(f"Expectation values: {pub_result.data.evs}")

    print(session.session_id)


In [13]:
#load results from existing file

resultList=[]
for i in range(16):
    with open("ExperimentData/result"+str(i)+".json", "r") as file:
        job_result = json.load(file, cls=RuntimeDecoder)
    resultList.append(job_result[0].data.evs)
    for idx, pub_result in enumerate(job_result):
        print(f"Expectation values for the circuit {i}: {pub_result.data.evs}")
 
dSPs=DspList_from_results(resultList)
dMs=DmList_from_results(resultList,dSPs)
print(f"Delta SP for the two qubits: {dSPs}")
print(f"Delta M for the two qubits: {dMs}")

Expectation values for the circuit 0: [0.95276583 0.98037332]
Expectation values for the circuit 1: [0.94285054 0.95623218]
Expectation values for the circuit 2: [0.95174771 0.98098127]
Expectation values for the circuit 3: [0.95341957 0.97910439]
Expectation values for the circuit 4: [-0.93177996 -0.96454112]
Expectation values for the circuit 5: [-0.93453694 -0.96723452]
Expectation values for the circuit 6: [-0.93730249 -0.96621901]
Expectation values for the circuit 7: [-0.93613275 -0.96624912]
Expectation values for the circuit 8: [0.97749325 0.96284007]
Expectation values for the circuit 9: [0.9759142  0.96243732]
Expectation values for the circuit 10: [0.97569883 0.96165072]
Expectation values for the circuit 11: [0.97596171 0.9630749 ]
Expectation values for the circuit 12: [-0.95737989 -0.94447057]
Expectation values for the circuit 13: [-0.96094815 -0.93911198]
Expectation values for the circuit 14: [-0.95715199 -0.9383447 ]
Expectation values for the circuit 15: [-0.95609325

In [16]:
import qiskit
qiskit.__version__

'1.2.0'